In [1]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device}.')

Using cuda.


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn

In [3]:
# You don't need to understand this function for now.
def load_data_CIFAR10(batch_size, resize=None):
    """Download the CIFAR10 dataset and then load it into memory."""
    trans = [torchvision.transforms.ToTensor()]
    if resize:
        trans.insert(0, torchvision.transforms.Resize(resize))
    trans = torchvision.transforms.Compose(trans)
    mnist_train = torchvision.datasets.CIFAR10(
        root="../data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.CIFAR10(
        root="../data", train=False, transform=trans, download=True)
    return (torch.utils.data.DataLoader(mnist_train, batch_size, shuffle=True,
                            num_workers=2),
            torch.utils.data.DataLoader(mnist_test, batch_size, shuffle=False,
                            num_workers=2))

In [4]:
batch_size = 64 # Defines the batch size
train_iter, test_iter = load_data_CIFAR10(batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
X, y = next(iter(train_iter)) # Requests the first training batch
print(X.size()) # 256 images per batch. Each image is represented by a 1 x 28 x 28 tensor (number of channels x height x width). The images are grayscale, so there is a single channel.
print(y.size())

torch.Size([64, 3, 32, 32])
torch.Size([64])


In [ ]:
class BlockGen(nn.Module):
    # Creating Intermediate Blocks
    def __init__(self)

In [6]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        
        self.fc = nn.Linear(out_channels, 1)
        
    def forward(self, x):
        out = self.relu(x)
        out1 = self.conv1(out)
        out2 = self.conv2(out)
        out3 = self.conv3(out)
        out4 = self.conv4(out)
        out5 = self.conv5(out)
        
        # Average pooling
        avg_pool = x.
        avg_pool = avg_pool.view(avg_pool.size(0), -1)
        # Weighted average calculation
        weights = torch.sigmoid(self.fc(avg_pool))
        print(weights.type())
        # Weighted sum
        weighted_sum = (out1 + out2 + out3 + out4 + out5) * weights.view(-1,1,1,1)
        
        return weighted_sum

In [8]:
# Applies Xavier initialization if the `torch.nn.Module` is `torch.nn.Linear` or `torch.nn.Conv2d`
def init_weights(m):
    if type(m) == torch.nn.Linear or type(m) == torch.nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)

num_outputs = 10
model = ConvBlock(in_channels=3, out_channels=num_outputs)
model.apply(init_weights) # Applies `init_weights` to every `torch.nn.Module` inside `model`

ConvBlock(
  (relu): ReLU()
  (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc): Linear(in_features=10, out_features=1, bias=True)
)

In [9]:
loss = torch.nn.CrossEntropyLoss()

In [10]:
lr = 0.9
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [11]:
def correct(logits, y):
    y_hat = logits.argmax(axis=1) # Finds the column with the highest value for each row of `logits`.
    return (y_hat == y).float().sum() # Computes the number of times that `y_hat` and `y` match.

In [12]:
def evaluate_metric(model, data_iter, metric):
    """Compute the average `metric` of the model on a dataset."""
    c = torch.tensor(0.)
    n = torch.tensor(0.)
    for X, y in data_iter:
        logits = model(X)
        c += metric(logits, y)
        n += len(y)

    return c / n

In [ ]:
print(f'Training accuracy: {evaluate_metric(model, train_iter, correct)}. Testing accuracy: {evaluate_metric(model, test_iter, correct)}.')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        
        self.fc = nn.Linear(out_channels, 10)  # Output size is 10
        
    def forward(self, x):
        out1 = F.relu(self.conv1(x))
        out2 = F.relu(self.conv2(out1))
        out3 = F.relu(self.conv3(out2))
        out4 = F.relu(self.conv4(out3))
        out5 = F.relu(self.conv5(out4))
        
        # Average pooling
        avg_pool = F.avg_pool2d(out5, kernel_size=out5.size()[2:])
        avg_pool = avg_pool.view(avg_pool.size(0), -1)
        
        # Weighted average calculation
        weights = torch.sigmoid(self.fc(avg_pool))
        
        # Weighted sum
        weighted_sum = (out1 + out2 + out3 + out4 + out5) * weights.view(-1, 10, 1, 1)  # Adjust the shape
        
        return weighted_sum

# Example usage
if __name__ == "__main__":
    # Assuming input image size is 32x32 and 3 channels
    input_image = torch.randn(64, 3, 32, 32)
    model = ConvBlock(in_channels=3, out_channels=64)
    output = model(input_image)
    print("Output size:", output.size())
